In [1]:
# importing necessary modules.
import requests
import json
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
# import config9 as cfg

In [7]:
postgress_password = ''
database_name = ''

In [3]:
# importing csv that contains the stock ticker symbols and names of the S&P 500.
sandp_df = pd.read_csv('../Resources/tickers_SandP_500.csv')

In [4]:
# creating empty list.
ticker_symbols = []

# looping through above csv and appending stock ticker symbols to the empty list.
for ticker in sandp_df['Ticker']:
    ticker_symbols.append(ticker)


# creating empty list.
names = []  

# looping through csv and appending names to the empty list.
for name in sandp_df['Name']:
    names.append(name)

In [5]:
# creating/initializing an empty pandas dataframe.
dataframe2 = pd.DataFrame()

# creating loop that goes through the ticker_symbols and names lists created above.
for (ticker, name) in zip(ticker_symbols, names):
    
    # creating a for loop that goes through the last two entry's value of the json dictionaries from financialmodelingprep.    
    for i in range(-2,0):
        
        # setting up try and except statement to avoid lack of financialmodelingprep json data to ticker_symbol list.         
        try:
            
            # obtaining the json data for financial ratios from financialmodelingprep.          
            response = requests.get(f'https://financialmodelingprep.com/api/v3/financial-ratios/{ticker}').json()
            
            print(f'Processing:{ticker} {name}')
            
            # creating empty list named columns.             
            columns = []
            
            # appending 'symbol' to columns list.                
            columns.append('symbol')
            
            # appending 'name' to columns list.            
            columns.append('name')
        
            # appending 'date' to columns list.         
            columns.append('date')
            
            # creating empty list named data.            
            data = []
            
            # filtering the the financial ratios json dicionary data by years 2009 and 2010.            
            if response['ratios'][i]['date'].split('-')[0] == '2009' or response['ratios'][i]['date'].split('-')[0] == '2010':
                
                # creating another try except statement to avoid pitfalls.                
                try:
                    
                    # appending symbol to data list.
                    data.append(response['symbol'])
                   
                    # appending company name to data list.
                    data.append(name)
                    
                    # appending date to data list.
                    data.append(response['ratios'][i]['date'])
                    
                    # saving financial ratio dicionary to variable.
                    keys = response['ratios'][i]
                    
                    # looping through finanial ratio keys.                   
                    for key in keys:
                        
                        # avoiding date key.                       
                        if key != 'date':
                            
                            # looping through finanial ratio dictionary.                            
                            for x in keys[key]:
                                
                                #  appending financial ratio dict keys to columns list.                                
                                columns.append(x)
                                
                                # filtering financial ratio values that are blank.                 
                                if keys[key][x] == '':
                                    
                                    # appending 0 in the spot of nothing to the data list.                                     
                                    data.append(0)
                                
                                # continuing the filtering process, but with some values.         
                                else:
                                    
                                    # appending those financial ratio values to data list.                                 
                                    data.append(float(keys[key][x]))
                    
                    # creating a connection and obtaining financial growth statistics from financialmodelingprep.
                    response2 = requests.get(f'https://financialmodelingprep.com/api/v3/financial-statement-growth/{ticker}').json()
                    
                    # saving financial growth dict to variable.                  
                    keys2 = response2['growth'][i]
                    
                    # creating for loop to go through financial grow. 
                    for key in keys2:
                        
                        # avoiding the date key.                       
                        if key != 'date':
                            
                            # appending the financial growth keys to columns list.                            
                            columns.append(key)
                            
                            # filtering financial growth values that have nan values.
                            if keys2[key] == '':

                                # replacing nan values with 0.                                 
                                data.append(0)

                            # filtering financial growth values that are not nan.
                            else:

                                # appending those financial growth values to the data list.        
                                data.append(float(keys2[key]))
                    
                    # creating a connection to get the company key metrics data from financialmodelingprep and putting in json form.
                    response3 = requests.get(f'https://financialmodelingprep.com/api/v3/enterprise-value/{ticker}').json()
                    
                    # filtering share price values that are nan.
                    if response3['enterpriseValues'][i]['Stock Price'] == '':
                        
                        # setting returns variable to 0 based on no share price data.                    
                        returns = 0
                    
                    # filtering chare price based on having a value.
                    else:
                        
                        # using share price of 2009 and 2018/2019 to calculated percentage return over the time span.                        
                        returns = ((((float(response3['enterpriseValues'][0]['Stock Price']) + float(response3['enterpriseValues'][1]['Stock Price']))/2) - ((float(response3['enterpriseValues'][-2]['Stock Price']) + float(response3['enterpriseValues'][-1]['Stock Price']))/2))/((float(response3['enterpriseValues'][-2]['Stock Price']) + float(response3['enterpriseValues'][-1]['Stock Price']))/2))*100
                    
                    # creating a pandas dataframe based on the data and column lists developed through the above process.
                    dataframe = pd.DataFrame([data], columns=columns)

                    # appending a share price return on investment to newly created dataframe.
                    dataframe['% return on investment'] = [returns]
                    
                    # appending the dataframe to the empty dataframe created at the very top.
                    dataframe2 = dataframe2.append(dataframe)
                
                # finishing try except statement.
                except:
                    
                    # telling program to move on and ignore pitfall.                    
                    pass
            
            # finishing date(2009,2010) if then statement created near the bottom of the top.         
            else:
                
                 # telling program to move on and ignore pitfall.
                pass
        
        # finishing the other try except statement.        
        except:
            
             # telling program to move on and ignore pitfall.
            pass

# grouping completed dataframe by symbol and name and finding the average of the columns, because used multiple dates.
dataframe2 = dataframe2.groupby(['symbol', 'name']).mean()

# viewing dataframe.
dataframe2

Processing:A Agilent Technologies, Inc.
Processing:A Agilent Technologies, Inc.
Processing:AAL American Airlines Group, Inc.
Processing:AAL American Airlines Group, Inc.
Processing:AAP Advance Auto Parts, Inc.
Processing:AAP Advance Auto Parts, Inc.
Processing:AAPL Apple, Inc.
Processing:AAPL Apple, Inc.
Processing:ABBV AbbVie, Inc.
Processing:ABBV AbbVie, Inc.
Processing:ABC AmerisourceBergen Corp.
Processing:ABC AmerisourceBergen Corp.
Processing:ABMD ABIOMED, Inc.
Processing:ABMD ABIOMED, Inc.
Processing:ABT Abbott Laboratories
Processing:ABT Abbott Laboratories
Processing:ACN Accenture Plc
Processing:ACN Accenture Plc
Processing:ADBE Adobe, Inc.
Processing:ADBE Adobe, Inc.
Processing:ADI Analog Devices, Inc.
Processing:ADI Analog Devices, Inc.
Processing:ADM Archer-Daniels-Midland Co.
Processing:ADM Archer-Daniels-Midland Co.
Processing:ADP Automatic Data Processing, Inc.
Processing:ADP Automatic Data Processing, Inc.
Processing:ADS Alliance Data Systems Corp.
Processing:ADS Allian

Processing:COO The Cooper Cos., Inc.
Processing:COP ConocoPhillips
Processing:COP ConocoPhillips
Processing:COST Costco Wholesale Corp.
Processing:COST Costco Wholesale Corp.
Processing:COTY Coty, Inc.
Processing:COTY Coty, Inc.
Processing:CPB Campbell Soup Co.
Processing:CPB Campbell Soup Co.
Processing:CPRI Capri Holdings Ltd.
Processing:CPRI Capri Holdings Ltd.
Processing:CPRT Copart, Inc.
Processing:CPRT Copart, Inc.
Processing:CRM salesforce.com, inc.
Processing:CRM salesforce.com, inc.
Processing:CSCO Cisco Systems, Inc.
Processing:CSCO Cisco Systems, Inc.
Processing:CSX CSX Corp.
Processing:CSX CSX Corp.
Processing:CTAS Cintas Corp.
Processing:CTAS Cintas Corp.
Processing:CTL CenturyLink, Inc.
Processing:CTL CenturyLink, Inc.
Processing:CTSH Cognizant Technology Solutions Corp.
Processing:CTSH Cognizant Technology Solutions Corp.
Processing:CTVA Corteva, Inc.
Processing:CTVA Corteva, Inc.
Processing:CTXS Citrix Systems, Inc.
Processing:CTXS Citrix Systems, Inc.
Processing:CVS CV

Processing:HON Honeywell International, Inc.
Processing:HP Helmerich & Payne, Inc.
Processing:HP Helmerich & Payne, Inc.
Processing:HPE Hewlett-Packard Enterprise Co.
Processing:HPE Hewlett-Packard Enterprise Co.
Processing:HPQ HP, Inc.
Processing:HPQ HP, Inc.
Processing:HRB H&R Block, Inc.
Processing:HRB H&R Block, Inc.
Processing:HRL Hormel Foods Corp.
Processing:HRL Hormel Foods Corp.
Processing:HSIC Henry Schein, Inc.
Processing:HSIC Henry Schein, Inc.
Processing:HST Host Hotels & Resorts, Inc.
Processing:HST Host Hotels & Resorts, Inc.
Processing:HSY The Hershey Co.
Processing:HSY The Hershey Co.
Processing:HUM Humana, Inc.
Processing:HUM Humana, Inc.
Processing:IBM International Business Machines Corp.
Processing:IBM International Business Machines Corp.
Processing:ICE Intercontinental Exchange, Inc.
Processing:ICE Intercontinental Exchange, Inc.
Processing:IDXX IDEXX Laboratories, Inc.
Processing:IDXX IDEXX Laboratories, Inc.
Processing:IEX IDEX Corp.
Processing:IEX IDEX Corp.
P

Processing:NRG NRG Energy, Inc.
Processing:NRG NRG Energy, Inc.
Processing:NSC Norfolk Southern Corp.
Processing:NSC Norfolk Southern Corp.
Processing:NTAP NetApp, Inc.
Processing:NTAP NetApp, Inc.
Processing:NTRS Northern Trust Corp.
Processing:NTRS Northern Trust Corp.
Processing:NUE Nucor Corp.
Processing:NUE Nucor Corp.
Processing:NVDA NVIDIA Corp.
Processing:NVDA NVIDIA Corp.
Processing:NVR NVR, Inc.
Processing:NVR NVR, Inc.
Processing:NWL Newell Brands, Inc.
Processing:NWL Newell Brands, Inc.
Processing:NWS News Corp.
Processing:NWS News Corp.
Processing:NWSA News Corp.
Processing:NWSA News Corp.
Processing:O Realty Income Corp.
Processing:O Realty Income Corp.
Processing:OKE ONEOK, Inc.
Processing:OKE ONEOK, Inc.
Processing:OMC Omnicom Group, Inc.
Processing:OMC Omnicom Group, Inc.
Processing:ORCL Oracle Corp.
Processing:ORCL Oracle Corp.
Processing:ORLY O'Reilly Automotive, Inc.
Processing:ORLY O'Reilly Automotive, Inc.
Processing:OXY Occidental Petroleum Corp.
Processing:OXY O

Processing:UPS United Parcel Service, Inc.
Processing:URI United Rentals, Inc.
Processing:URI United Rentals, Inc.
Processing:USB U.S. Bancorp
Processing:USB U.S. Bancorp
Processing:UTX United Technologies Corp.
Processing:UTX United Technologies Corp.
Processing:V Visa, Inc.
Processing:V Visa, Inc.
Processing:VAR Varian Medical Systems, Inc.
Processing:VAR Varian Medical Systems, Inc.
Processing:VFC VF Corp.
Processing:VFC VF Corp.
Processing:VIAB Viacom Inc.
Processing:VIAB Viacom Inc.
Processing:VLO Valero Energy Corp.
Processing:VLO Valero Energy Corp.
Processing:VMC Vulcan Materials Co.
Processing:VMC Vulcan Materials Co.
Processing:VNO Vornado Realty Trust
Processing:VNO Vornado Realty Trust
Processing:VRSK Verisk Analytics, Inc.
Processing:VRSK Verisk Analytics, Inc.
Processing:VRSN VeriSign, Inc.
Processing:VRSN VeriSign, Inc.
Processing:VRTX Vertex Pharmaceuticals, Inc.
Processing:VRTX Vertex Pharmaceuticals, Inc.
Processing:VTR Ventas, Inc.
Processing:VTR Ventas, Inc.
Process

,,priceBookValueRatio,priceToBookRatio,priceToSalesRatio,priceEarningsRatio,receivablesTurnover,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,...,5Y Dividend per Share Growth (per Share),3Y Dividend per Share Growth (per Share),Receivables growth,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,% return on investment
symbol,name,,,,,,,,,,,,,,,,,,,,,
A,"Agilent Technologies, Inc.",2.55170,5.17165,1.48130,6.30885,7.00140,18.18070,13.57995,0.0,0.0,0.0,...,0.00000,0.00000,0.11660,0.07580,0.18005,0.15185,0.31885,-0.06740,0.01880,215.407785
AAL,"American Airlines Group, Inc.",0.00000,0.00000,0.11565,0.00000,27.33480,0.00000,2.20925,0.0,0.0,0.0,...,0.00000,0.00000,-0.04605,0.06370,-0.00170,-0.06235,0.00975,0.00000,0.01435,492.976589
AAP,"Advance Auto Parts, Inc.",2.95660,3.10380,0.70600,14.16840,57.04610,7.41850,5.46130,0.0,0.0,0.0,...,0.00000,0.00000,-0.04780,0.00540,0.03670,0.19520,-0.57150,0.00000,0.10360,158.290852
AAPL,"Apple, Inc.",5.42765,4.62715,3.98235,19.54495,10.09060,17.32790,15.35960,0.0,0.0,0.0,...,0.00000,0.00000,1.02515,0.60190,0.44800,0.44245,0.00000,0.26940,0.21645,641.441749
ABBV,"AbbVie, Inc.",16.46870,0.00000,3.01900,10.48360,0.00000,9.21230,8.74540,0.0,0.0,0.0,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.46130,0.14080,127.493713
ABC,AmerisourceBergen Corp.,2.65245,39.54195,0.10130,13.48380,19.74495,9.91215,8.09975,0.0,0.0,0.0,...,0.59785,0.17525,0.05130,0.11420,0.08720,0.11490,0.06560,0.00000,0.01405,258.118955
ABMD,"ABIOMED, Inc.",3.53820,5.63330,4.51390,0.00000,5.86270,0.00000,-52.55390,0.0,0.0,0.0,...,0.00000,0.00000,-0.14040,-0.37670,-0.04700,-0.11940,0.00000,0.02470,0.09900,2583.734150
ABT,Abbott Laboratories,1.64610,5.42750,1.15640,7.34805,5.12415,5.54100,4.78745,0.0,0.0,0.0,...,-0.03630,0.10510,0.14755,0.07645,0.19580,0.15215,0.29370,0.18890,0.11545,167.453799
ACN,Accenture Plc,7.75270,6.74855,1.01325,13.02975,6.11045,7.59970,7.49890,0.0,0.0,0.0,...,0.00000,0.33070,-0.08325,0.00000,0.01790,0.03930,-0.11090,0.00000,0.02345,363.726903


In [6]:
# dropping columns with little to no data.
dataframe2 = dataframe2.drop(columns=['priceCashFlowRatio', 'priceEarningsToGrowthRatio', 'priceSalesRatio', 'dividendYield', 'priceFairValue', 'operatingProfitMargin'])

In [7]:
# filtering the completed dataframe by creating a new dataframe that the does not have the rows associated with 0 in the % return on investment column.
sandp = dataframe2[dataframe2['% return on investment'] != 0]

In [8]:
# replacing all the 0 values with -99 in the dataframes for the purpose of machine learning to indicate nan.
sandp  = sandp.replace(0,-99)

In [9]:
# changing all the column names for the purpose of sql.
sandp = sandp.rename(columns = {'priceBookValueRatio': 'price_to_bookvalue', 'priceToBookRatio': 'price_to_book',
           'priceToSalesRatio': 'price_to_sales', 'priceEarningsRatio': 'price_to_earnings', 'receivablesTurnover': 'receivables_turnover',
           'priceToFreeCashFlowsRatio': 'price_to_free_cash_flow', 'priceToOperatingCashFlowsRatio': 'price_to_operating_cash_flow',
           'enterpriseValueMultiple': 'enterprise_value_multiple', 'niperEBT': 'niper_ebt', 'ebtperEBIT': 'ebt_per_ebit', 'ebitperRevenue': 'ebit_per_revenue',
           'grossProfitMargin': 'gross_profit_margin', 'pretaxProfitMargin': 'pretax_profit_margin', 'netProfitMargin': 'net_profit_margin',
           'effectiveTaxRate': 'tax_rate', 'returnOnAssets': 'return_on_assets', 'returnOnEquity': 'return_on_equity',
           'returnOnCapitalEmployed': 'return_on_capital_employed', 'nIperEBT': 'niper_ebt', 'eBTperEBIT': 'ebt_per_ebit', 'eBITperRevenue': 'ebit_per_revenue',
           'receivablesTurnover.1': 'receivables_turnover_1', 'payablesTurnover': 'payables_turnover', 'inventoryTurnover': 'inventory_turnover',
           'fixedAssetTurnover': 'fixed_asset_turnover', 'assetTurnover': 'asset_turnover', 'currentRatio': 'current_ratio', 'quickRatio': 'quick_ratio',
           'cashRatio': 'cash_ratio', 'daysOfSalesOutstanding': 'days_of_sales_outstanding', 'daysOfInventoryOutstanding': 'days_of_inventory_outstanding',
           'operatingCycle': 'operating_cycle', 'daysOfPayablesOutstanding': 'days_of_payables_outstanding', 'cashConversionCycle': 'cash_conversion_cycle',
           'debtRatio': 'debt_ratio', 'debtEquityRatio':'debt_to_equity_ratio', 'longtermDebtToCapitalization': 'longterm_debt_to_capitalization',
           'totalDebtToCapitalization': 'total_debt_to_capitalization', 'interestCoverage': 'interest_coverage', 'cashFlowToDebtRatio': 'cash_flow_to_debt',
           'companyEquityMultiplier': 'company_equity_multiplier', 'operatingCashFlowPerShare': 'operating_cash_flow_per_share',
           'freeCashFlowPerShare': 'free_cash_flow_per_share', 'cashPerShare': 'cash_per_share', 'payoutRatio': 'payout_ratio',
           'receivablesTurnover.2': 'receivables_turnover_2', 'operatingCashFlowSalesRatio': 'operating_cash_flow_to_sales',
           'freeCashFlowOperatingCashFlowRatio': 'free_cash_flow_to_operating_cash_flow', 'cashFlowCoverageRatios': 'cash_flow_coverage_ratio',
           'shortTermCoverageRatios': 'short_term_coverage_ratio', 'capitalExpenditureCoverageRatios': 'capital_expenditure_coverage_ratio',
           'dividendpaidAndCapexCoverageRatios': 'dividend_capex_coverage_ratio', 'dividendPayoutRatio': 'dividend_payout_ratio',
           'Gross Profit Growth': 'gross_profit_growth', 'EBIT Growth': 'ebit_growth', 'Operating Income Growth': 'operating_income_growth',
           'Net Income Growth': 'net_income_growth', 'EPS Growth': 'eps_growth', 'EPS Diluted Growth': 'eps_diluted_growth',
           'Weighted Average Shares Growth': 'weighted_average_share_growth',
           'Weighted Average Shares Diluted Growth': 'weighted_averages_shares_diluted_growth', 'Dividends per Share Growth': 'dividents_per_share_growth',
           'Operating Cash Flow growth': 'operating_cash_flow_growth', 'Free Cash Flow growth': 'free_cash_flow_growth',
           '10Y Revenue Growth (per Share)': 'ten_year_revenue_growth_per_share', '5Y Revenue Growth (per Share)': 'five_year_revenue_growth_per_share',
           '3Y Revenue Growth (per Share)': 'three_year_revenue_growth_per_share', '10Y Operating CF Growth (per Share)': 'ten_year_operating_cf_growth_per_share',
           '5Y Operating CF Growth (per Share)': 'five_year_operating_cf_growth_per_share',
           '3Y Operating CF Growth (per Share)': 'three_year_operating_cf_growth_per_share',
           '10Y Net Income Growth (per Share)': 'ten_year_net_income_growth_per_share', '5Y Net Income Growth (per Share)': 'five_year_net_income_growth_per_share',
           '3Y Net Income Growth (per Share)': 'three_year_net_income_growth_per_share',
           '10Y Shareholders Equity Growth (per Share)':'ten_year_shareholders_equity_growth_per_share',
           '5Y Shareholders Equity Growth (per Share)':'five_year_shareholders_equity_growth_per_share',
           '3Y Shareholders Equity Growth (per Share)':'three_year_shareholders_equity_growth_per_share',
           '10Y Dividend per Share Growth (per Share)':'ten_year_dividend_growth_per_share',
           '5Y Dividend per Share Growth (per Share)':'five_year_dividend_growth_per_share',
           '3Y Dividend per Share Growth (per Share)':'three_year_dividend_growth_per_share', 'Receivables growth': 'receivables_growh',
           'Inventory Growth': 'inventory_growth', 'Asset Growth': 'asset_growth', 'Book Value per Share Growth': 'book_value_per_share_growth',
           'Debt Growth': 'debt_growth', 'R&D Expense Growth':'randd_expense_growth', 'SG&A Expenses Growth': 'sganda_expense_growth',
           '% return on investment': 'percent_return_on_investment'})

In [10]:
# dropping all unecessary columns.
sandp = sandp.drop(columns=['niper_ebt', 'ebt_per_ebit',  'ebit_per_revenue'])

In [20]:
# reseting the index of the sandp dataframe.
sandp = sandp.reset_index()
sandp

,index,symbol,name,price_to_bookvalue,price_to_book,price_to_sales,price_to_earnings,receivables_turnover,price_to_free_cash_flow,price_to_operating_cash_flow,...,five_year_dividend_growth_per_share,three_year_dividend_growth_per_share,receivables_growh,inventory_growth,asset_growth,book_value_per_share_growth,debt_growth,randd_expense_growth,sganda_expense_growth,percent_return_on_investment
0,0,A,"Agilent Technologies, Inc.",2.55170,5.17165,1.48130,6.30885,7.00140,18.18070,13.57995,...,-99.00000,-99.00000,0.11660,0.07580,0.18005,0.15185,0.31885,-0.06740,0.01880,215.407785
1,1,AAL,"American Airlines Group, Inc.",-99.00000,-99.00000,0.11565,-99.00000,27.33480,-99.00000,2.20925,...,-99.00000,-99.00000,-0.04605,0.06370,-0.00170,-0.06235,0.00975,-99.00000,0.01435,492.976589
2,2,AAP,"Advance Auto Parts, Inc.",2.95660,3.10380,0.70600,14.16840,57.04610,7.41850,5.46130,...,-99.00000,-99.00000,-0.04780,0.00540,0.03670,0.19520,-0.57150,-99.00000,0.10360,158.290852
3,3,AAPL,"Apple, Inc.",5.42765,4.62715,3.98235,19.54495,10.09060,17.32790,15.35960,...,-99.00000,-99.00000,1.02515,0.60190,0.44800,0.44245,-99.00000,0.26940,0.21645,641.441749
4,4,ABBV,"AbbVie, Inc.",16.46870,-99.00000,3.01900,10.48360,-99.00000,9.21230,8.74540,...,-99.00000,-99.00000,-99.00000,-99.00000,-99.00000,-99.00000,-99.00000,0.46130,0.14080,127.493713
5,5,ABC,AmerisourceBergen Corp.,2.65245,39.54195,0.10130,13.48380,19.74495,9.91215,8.09975,...,0.59785,0.17525,0.05130,0.11420,0.08720,0.11490,0.06560,-99.00000,0.01405,258.118955
6,6,ABMD,"ABIOMED, Inc.",3.53820,5.63330,4.51390,-99.00000,5.86270,-99.00000,-52.55390,...,-99.00000,-99.00000,-0.14040,-0.37670,-0.04700,-0.11940,-99.00000,0.02470,0.09900,2583.734150
7,7,ABT,Abbott Laboratories,1.64610,5.42750,1.15640,7.34805,5.12415,5.54100,4.78745,...,-0.03630,0.10510,0.14755,0.07645,0.19580,0.15215,0.29370,0.18890,0.11545,167.453799
8,8,ACN,Accenture Plc,7.75270,6.74855,1.01325,13.02975,6.11045,7.59970,7.49890,...,-99.00000,0.33070,-0.08325,-99.00000,0.01790,0.03930,-0.11090,-99.00000,0.02345,363.726903
9,9,ADBE,"Adobe, Inc.",3.28150,17.69555,5.09575,33.68390,7.29175,17.04700,14.94845,...,-1.00000,-99.00000,0.11425,-99.00000,0.18440,0.10660,1.18980,0.02870,0.08435,750.511115


In [22]:
# creating an empty list.
ticker_symbols = []
# looping through the ticker symbol column of dataframe and appending to the empty list.
for symbol in sandp['symbol']:
    ticker_symbols.append(symbol)

In [23]:
# creating empty list.
industry_list = []

# creating empty list.
sector_list = []

# looping through ticker_symbols list.
for ticker in ticker_symbols:
    
    # try except statement to ignore missing data.    
    try:

        # obtaining the json data for sector and industry type from financialmodelingprep.          
        response = requests.get(f'https://financialmodelingprep.com/api/v3/company/profile/{ticker}').json()

        # getting industry data.
        industry_list.append(response['profile']['industry'])
        
        # getting sector data.
        sector_list.append(response['profile']['sector'])
    
    # try except statement to ignore missing data.
    except:
        pass

In [24]:
# adding industry to dataframe.
sandp['industry'] = industry_list

In [25]:
# adding sector to dataframe.
sandp['sector'] = sector_list

In [32]:
# calculating the total returns for all the companies in the dataframe the 10 year span 09/10-18/19
total_percent = sandp['percent_return_on_investment'].sum()
total_return = (total_percent - len(sandp['percent_return_on_investment']))/len(sandp['percent_return_on_investment'])

In [34]:
# creating empty list.
above_below_sandp = []

# looping through percent_return_on_investment column.
for returns in sandp['percent_return_on_investment']:
    # finding returns that are above 228 and appending 1 the empty list.      
    if returns > total_return:
        above_below_sandp.append(1)
    
    #  appending 0 to the empty list with 0 for percent returns under 228.    
    else:
        above_below_sandp.append(0)

# using the appended list created above and adding it as a column to the dataframe.
sandp['above_below_sandp_return'] = above_below_sandp 

In [35]:
# verifying the above_below_sandp column was added.
sandp.head()

,index,symbol,name,price_to_bookvalue,price_to_book,price_to_sales,price_to_earnings,receivables_turnover,price_to_free_cash_flow,price_to_operating_cash_flow,...,inventory_growth,asset_growth,book_value_per_share_growth,debt_growth,randd_expense_growth,sganda_expense_growth,percent_return_on_investment,industry,sector,above_below_sandp_return
0,0,A,"Agilent Technologies, Inc.",2.55170,5.17165,1.48130,6.30885,7.0014,18.1807,13.57995,...,0.0758,0.18005,0.15185,0.31885,-0.0674,0.01880,215.407785,Medical Diagnostics & Research,Healthcare,0
1,1,AAL,"American Airlines Group, Inc.",-99.00000,-99.00000,0.11565,-99.00000,27.3348,-99.0000,2.20925,...,0.0637,-0.00170,-0.06235,0.00975,-99.0000,0.01435,492.976589,Airlines,Industrials,1
2,2,AAP,"Advance Auto Parts, Inc.",2.95660,3.10380,0.70600,14.16840,57.0461,7.4185,5.46130,...,0.0054,0.03670,0.19520,-0.57150,-99.0000,0.10360,158.290852,Retail - Apparel & Specialty,Consumer Cyclical,0
3,3,AAPL,"Apple, Inc.",5.42765,4.62715,3.98235,19.54495,10.0906,17.3279,15.35960,...,0.6019,0.44800,0.44245,-99.00000,0.2694,0.21645,641.441749,Computer Hardware,Technology,1
4,4,ABBV,"AbbVie, Inc.",16.46870,-99.00000,3.01900,10.48360,-99.0000,9.2123,8.74540,...,-99.0000,-99.00000,-99.00000,-99.00000,0.4613,0.14080,127.493713,Drug Manufacturers,Healthcare,0


In [36]:
# creating a connection to an aws rds postgres cloud database.
rds_connection_string = f'postgres:{postgress_password}@localhost:5432/{database_name}'
engine = create_engine(f'postgres://{rds_connection_string}')

In [38]:
# checking out the table names in the cloud database.
engine.table_names()

[]

In [39]:
# appedending pandas dataframe to cloud postgres database.
sandp.to_sql(name='sandp', con=engine, if_exists="replace", index=False)

In [40]:
# checking to to see if above process worked.
pd.read_sql_query('SELECT * FROM sandp', con=engine).head()

,index,symbol,name,price_to_bookvalue,price_to_book,price_to_sales,price_to_earnings,receivables_turnover,price_to_free_cash_flow,price_to_operating_cash_flow,...,inventory_growth,asset_growth,book_value_per_share_growth,debt_growth,randd_expense_growth,sganda_expense_growth,percent_return_on_investment,industry,sector,above_below_sandp_return
0,0,A,"Agilent Technologies, Inc.",2.55170,5.17165,1.48130,6.30885,7.0014,18.1807,13.57995,...,0.0758,0.18005,0.15185,0.31885,-0.0674,0.01880,215.407785,Medical Diagnostics & Research,Healthcare,0
1,1,AAL,"American Airlines Group, Inc.",-99.00000,-99.00000,0.11565,-99.00000,27.3348,-99.0000,2.20925,...,0.0637,-0.00170,-0.06235,0.00975,-99.0000,0.01435,492.976589,Airlines,Industrials,1
2,2,AAP,"Advance Auto Parts, Inc.",2.95660,3.10380,0.70600,14.16840,57.0461,7.4185,5.46130,...,0.0054,0.03670,0.19520,-0.57150,-99.0000,0.10360,158.290852,Retail - Apparel & Specialty,Consumer Cyclical,0
3,3,AAPL,"Apple, Inc.",5.42765,4.62715,3.98235,19.54495,10.0906,17.3279,15.35960,...,0.6019,0.44800,0.44245,-99.00000,0.2694,0.21645,641.441749,Computer Hardware,Technology,1
4,4,ABBV,"AbbVie, Inc.",16.46870,-99.00000,3.01900,10.48360,-99.0000,9.2123,8.74540,...,-99.0000,-99.00000,-99.00000,-99.00000,0.4613,0.14080,127.493713,Drug Manufacturers,Healthcare,0
